# Black Scholes Exercise 5: Cython implementation

- Use cython
- Use cProfile and VTune to look for bottlenecks and hotspots in the code

In [1]:
#Boilerplate for the example

import cProfile
import pstats
import numpy as np
%load_ext Cython

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

# The Cython Black Scholes algorithm

In [2]:
%pycat cython_scholes.pyx

#!/usr/bin/env python
# Copyright (c) 2017, Intel Corporation
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright notice,
#       this list of conditions and the following disclaimer.
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#     * Neither the name of Intel Corporation nor the names of its contributors
#       may be used to endorse or promote products derived from this software
#       without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FO

You'll need to build from the .pyx file
- Run the following: python setup_scholes.py build_ext --inplace

In [3]:
!python setup_scholes.py build_ext --inplace

running build_ext


Now you should be able to import the built cython component

In [4]:
import cython_scholes as cbs

And run timeit.

In [5]:
%timeit cbs.black_scholes(nopt, price, strike, t, 0.1, 0.2, call, put)

10 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Use Intel compiler.

In [6]:
!gedit setup_scholes.py

/usr/bin/sh: gedit: command not found


In [7]:
!python setup_scholes.py build_ext --inplace --force

running build_ext
building 'cython_scholes' extension
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -march=x86-64 -mtune=generic -O2 -pipe -fstack-protector-strong -fno-plt -march=x86-64 -mtune=generic -O2 -pipe -fstack-protector-strong -fno-plt -march=x86-64 -mtune=generic -O2 -pipe -fstack-protector-strong -fno-plt -fPIC -I/home/pierre/.virtualenvs/ep2018/lib/python3.6/site-packages/numpy/core/include -I/usr/include/python3.6m -c cython_scholes.c -o build/temp.linux-x86_64-3.6/cython_scholes.o
In file included from /home/pierre/.virtualenvs/ep2018/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1818,
                 from /home/pierre/.virtualenvs/ep2018/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /home/pierre/.virtualenvs/ep2018/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from cython_scholes.c:580:
/home/pierre/.virtualenvs/ep2018/lib/pytho

## The reload machansim doesn't always work. Restart notebook and run again.

Use VTune

In [8]:
%load_ext cython

In [9]:
%cython

import numpy as np
cimport numpy as np

from cython cimport boundscheck, wraparound, cdivision, initializedcheck
from cython.parallel cimport prange, parallel
from scipy.special import erf as sp_erf
#from libc.stdlib cimport srand, rand, RAND_MAX

# For better performance, use icc
# cdef extern from "mathimf.h":
cdef extern from "math.h":
    double erf(double x) nogil
    double log(double x) nogil
    double exp(double x) nogil
    double sqrt(double x) nogil

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

# In order to release the GIL for a parallel loop, code in the with block cannot
# manipulate Python objects in any way.
@boundscheck(False)
@wraparound(False)
@cdivision(True)
@initializedcheck(False)
def black_scholes(int nopt,
                  double[:] price,
                  double[:] strike,
                  double[:] t,
                  double rate,
                  double vol,
                  double[:] call,
                  double[:] put):

    cdef int i
    cdef double P, S, a, b, z, c, Se, y, T
    cdef double d1, d2, w1, w2
    cdef double mr = -rate
    cdef double sig_sig_two = vol * vol * 2

    with nogil, parallel():
        for i in prange(nopt):
            P = price [i]
            S = strike [i]
            T = t [i]

            a = log(P / S)
            b = T * mr

            z = T * sig_sig_two
            c = 0.25 * z
            y = 1/sqrt(z)

            w1 = (a - b + c) * y
            w2 = (a - b - c) * y

            d1 = 0.5 + 0.5 * erf(w1)
            d2 = 0.5 + 0.5 * erf(w2)

            Se = exp(b) * S

            call [i] = P * d1 - Se * d2
            put [i] = call [i] - P + Se



SyntaxError: invalid syntax (<ipython-input-9-a47d484c4792>, line 4)